# Telegram Bot Client

## Install Dependencies

In [3]:
!python -m pip install -U python-telegram-bot matplotlib prettytable pandas

## Import Dependencies

In [5]:
from telegram import Update, InlineQueryResultArticle, InputTextMessageContent, ParseMode
from telegram.ext import Updater, CallbackContext, InlineQueryHandler, CommandHandler, MessageHandler, Filters
import io
import matplotlib.pyplot as plt
import prettytable as pt
import pandas as pd

## Implementation

In [2]:
class TelegramBot(object):
    def __init__(self, bot_token: str, register_handlers = None):
        assert bot_token is not None
        
        self.bot_token = bot_token
        self.register_handlers = register_handlers

    def __enter__(self):
        self.updater = Updater(token=self.bot_token)
        self.dispatcher = self.updater.dispatcher
        self.bot = self.updater.bot

        if self.register_handlers is not None:
            self.register_handlers(self.dispatcher)

        self.updater.start_polling()

        return self

    def __exit__(self, exc_type, exc_value, tb):
        self.updater.stop()
        self.updater = None
        self.dispatcher = None
        self.bot = None

    def __get_figure_buffer__(self, figure: plt.Figure):
        buffer = io.BytesIO()
        figure.savefig(buffer, format='png')
        buffer.seek(0)
        
        return buffer

    def send_figure(self, chat_id: str, figure: plt.Figure):
        self.bot.send_photo(chat_id=chat_id, photo=self.__get_figure_buffer__(figure))

    def send_dataframe(self, chat_id: str, dataframe: pd.DataFrame):
        table = pt.PrettyTable(tuple(dataframe.columns))
        
        for index, row in dataframe.iterrows():
            row_data = list()

            for column_name in dataframe.columns:
                row_data.append(row[column_name])
            
            table.add_row(row_data)

        self.bot.send_message(chat_id=chat_id, text=f'<pre>{table}</pre>', parse_mode=ParseMode.HTML)